In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from ipywidgets import widgets

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
%matplotlib inline

C:\Users\erinm\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
donations=pd.read_pickle('donations_cohort_with_cycle.pkl')

In [13]:
for col in donations.columns:
    print(col)

id
first_gift_year
lapsed_count
recovered_count
max_consec_giving_years
total_giving_years
count_2013
count_2014
count_2015
count_2016
count_2017
count_yr0
count_yr1
count_yr2
count_yr3
count_yr4
initial
first_gift
second_year
second_with_gap
multi_year
key_multi_year
recovered_multi_year
inconsistent_multi_year
recovered_key_multi_year
lapsed_after_first
lapsed_after_first_long
lapsed_after_second
lapsed_multi_year
lapsed_long_multi_year
lapsed_key_multi_year
lapsed_long
cohort_0_first_gift
cohort_1_lapsed_after_first
cohort_1_second_year
cohort_2_lapsed_after_first_long
cohort_2_lapsed_after_second
cohort_2_multi_year
cohort_2_second_with_gap
cohort_3_first_gift
cohort_3_inconsistent_multi_year
cohort_3_key_multi_year
cohort_3_lapsed_after_second
cohort_3_lapsed_long
cohort_3_lapsed_multi_year
cohort_4_first_gift
cohort_4_inconsistent_multi_year
cohort_4_key_multi_year
cohort_4_lapsed_after_first
cohort_4_lapsed_key_multi_year
cohort_4_lapsed_long
cohort_4_lapsed_long_multi_year
coho

In [14]:
donations.yr2_TF_accum.value_counts()

TFF    1724319
TTF     175947
TTT      67489
TFT      56799
Name: yr2_TF_accum, dtype: int64

### 1. Parallel categories

In [4]:
#restrict to one cohort
c13=donations.query('cohort=="cohort13"')

In [29]:
#restrict to only necessary columns
c13a=c13[['cohort', 'cohort_gift_pattern', 'yr0_TF', 'yr1_TF','yr2_TF','yr3_TF','yr4_TF','yr0_TF_accum', 'yr1_TF_accum','yr2_TF_accum','yr3_TF_accum','yr4_TF_accum','yr0_behavior', 'yr1_behavior', 'yr2_behavior', 'yr3_behavior', 'yr4_behavior','yr0_behavior2', 'yr1_behavior2', 'yr2_behavior2', 'yr3_behavior2', 'yr4_behavior2']].copy()

In [30]:
c13a

,cohort,cohort_gift_pattern,yr0_TF,yr1_TF,yr2_TF,yr3_TF,yr4_TF,yr0_TF_accum,yr1_TF_accum,yr2_TF_accum,...,yr0_behavior,yr1_behavior,yr2_behavior,yr3_behavior,yr4_behavior,yr0_behavior2,yr1_behavior2,yr2_behavior2,yr3_behavior2,yr4_behavior2
0,cohort13,"0,",T,F,F,F,F,T,TF,TFF,...,new,lapse,lapse_long,lapse_long,lapse_long,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
10,cohort13,"0,1,",T,T,F,F,F,T,TT,TTF,...,new,retain,lapse,lapse_long,lapse_long,n/a,maintain,did_not_donate,did_not_donate,did_not_donate
14,cohort13,"0,",T,F,F,F,F,T,TF,TFF,...,new,lapse,lapse_long,lapse_long,lapse_long,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
18,cohort13,"0,1,2,",T,T,T,F,F,T,TT,TTT,...,new,retain,retain,lapse,lapse_long,n/a,maintain,maintain,did_not_donate,did_not_donate
27,cohort13,"0,1,",T,T,F,F,F,T,TT,TTF,...,new,retain,lapse,lapse_long,lapse_long,n/a,downgrade,did_not_donate,did_not_donate,did_not_donate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024519,cohort13,"0,",T,F,F,F,F,T,TF,TFF,...,new,lapse,lapse_long,lapse_long,lapse_long,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2024520,cohort13,"0,2,",T,F,T,F,F,T,TF,TFT,...,new,lapse,recover,lapse,lapse_long,n/a,did_not_donate,upgrade,did_not_donate,did_not_donate
2024521,cohort13,"0,",T,F,F,F,F,T,TF,TFF,...,new,lapse,lapse_long,lapse_long,lapse_long,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2024522,cohort13,"0,",T,F,F,F,F,T,TF,TFF,...,new,lapse,lapse_long,lapse_long,lapse_long,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate


In [31]:
#keep only those that donated each year
c13b=c13a.query('yr1_behavior2 != "n/a" & yr2_behavior2 != "did_not_donate" & yr3_behavior2 != "did_not_donate" & yr4_behavior2 != "did_not_donate"')

In [32]:
c13b

,cohort,cohort_gift_pattern,yr0_TF,yr1_TF,yr2_TF,yr3_TF,yr4_TF,yr0_TF_accum,yr1_TF_accum,yr2_TF_accum,...,yr0_behavior,yr1_behavior,yr2_behavior,yr3_behavior,yr4_behavior,yr0_behavior2,yr1_behavior2,yr2_behavior2,yr3_behavior2,yr4_behavior2
41,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,upgrade,upgrade,downgrade,downgrade
107,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,downgrade,downgrade,upgrade,upgrade
320,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,downgrade,upgrade,upgrade,upgrade
374,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,upgrade,maintain,maintain,upgrade
688,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,downgrade,downgrade,downgrade,upgrade
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023707,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,downgrade,upgrade,downgrade,downgrade
2023937,cohort13,"0,2,3,4,",T,F,T,T,T,T,TF,TFT,...,new,lapse,recover,retain,retain,n/a,did_not_donate,upgrade,downgrade,upgrade
2023991,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,upgrade,downgrade,upgrade,downgrade
2024029,cohort13,"0,1,2,3,4,",T,T,T,T,T,T,TT,TTT,...,new,retain,retain,retain,retain,n/a,upgrade,downgrade,upgrade,downgrade


In [33]:
c13b.groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2',
              #'yr3_behavior2', 'yr4_behavior2'
             ]).count().reset_index()

,cohort,yr1_behavior2,yr2_behavior2,cohort_gift_pattern,yr0_TF,yr1_TF,yr2_TF,yr3_TF,yr4_TF,yr0_TF_accum,...,yr3_TF_accum,yr4_TF_accum,yr0_behavior,yr1_behavior,yr2_behavior,yr3_behavior,yr4_behavior,yr0_behavior2,yr3_behavior2,yr4_behavior2
0,cohort13,did_not_donate,downgrade,613,613,613,613,613,613,613,...,613,613,613,613,613,613,613,613,613,613
1,cohort13,did_not_donate,maintain,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
2,cohort13,did_not_donate,upgrade,1068,1068,1068,1068,1068,1068,1068,...,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068
3,cohort13,downgrade,downgrade,954,954,954,954,954,954,954,...,954,954,954,954,954,954,954,954,954,954
4,cohort13,downgrade,maintain,150,150,150,150,150,150,150,...,150,150,150,150,150,150,150,150,150,150
5,cohort13,downgrade,upgrade,2182,2182,2182,2182,2182,2182,2182,...,2182,2182,2182,2182,2182,2182,2182,2182,2182,2182
6,cohort13,maintain,downgrade,101,101,101,101,101,101,101,...,101,101,101,101,101,101,101,101,101,101
7,cohort13,maintain,maintain,137,137,137,137,137,137,137,...,137,137,137,137,137,137,137,137,137,137
8,cohort13,maintain,upgrade,323,323,323,323,323,323,323,...,323,323,323,323,323,323,323,323,323,323
9,cohort13,upgrade,downgrade,3443,3443,3443,3443,3443,3443,3443,...,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443


In [34]:
df=c13b
# Create dimensions
class_dim = go.parcats.Dimension(
    values=df.cohort_gift_pattern,
    categoryorder='category ascending', label=""
)

gender_dim = go.parcats.Dimension(values=df.yr1_behavior2, categoryorder='category descending', label="yr1")

gender1_dim = go.parcats.Dimension(values=df.yr2_behavior2, categoryorder='category descending', label="yr2")

gender2_dim = go.parcats.Dimension(values=df.yr3_behavior2, categoryorder='category descending', label="yr3")

gender3_dim = go.parcats.Dimension(values=df.yr4_behavior2, categoryorder='category descending', label="yr4")


#survival_dim = go.parcats.Dimension(
  #  values=df.behavior_yr2, label="yr2", 
 #   #categoryarray=[0, 1],
  #  ticktext=['perished', 'survived'])

# Create parcats trace
#color = df.behavior_yr1;
#colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, gender1_dim, gender2_dim, gender3_dim],
 #       line={'color': color, 'colorscale': colorscale},
      #  hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [58]:
df=donations.query('cohort=="cohort13"').groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2', 'yr3_behavior2', 'yr4_behavior2']).count().reset_index()
# Create dimensions
class_dim = go.parcats.Dimension(
    values=df.cohort,
    categoryorder='category ascending', label=""
)

gender_dim = go.parcats.Dimension(values=df.yr1_behavior2, categoryorder='category descending', label="yr1")

gender1_dim = go.parcats.Dimension(values=df.yr2_behavior2, categoryorder='category descending', label="yr2")

gender2_dim = go.parcats.Dimension(values=df.yr3_behavior2, categoryorder='category descending', label="yr3")

gender3_dim = go.parcats.Dimension(values=df.yr4_behavior2, categoryorder='category descending', label="yr4")


#survival_dim = go.parcats.Dimension(
  #  values=df.behavior_yr2, label="yr2", 
 #   #categoryarray=[0, 1],
  #  ticktext=['perished', 'survived'])

# Create parcats trace
#color = df.behavior_yr1;
#colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, gender1_dim, gender2_dim, gender3_dim],
 #       line={'color': color, 'colorscale': colorscale},
      #  hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [35]:
df=c13b.groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2','yr3_behavior2', 'yr4_behavior2']).count().reset_index()
# Create dimensions
class_dim = go.parcats.Dimension(
    values=df.cohort,
    categoryorder='category ascending', label=""
)

gender_dim = go.parcats.Dimension(values=df.yr1_behavior2, categoryorder='category descending', label="yr1")

gender1_dim = go.parcats.Dimension(values=df.yr2_behavior2, categoryorder='category descending', label="yr2")

gender2_dim = go.parcats.Dimension(values=df.yr3_behavior2, categoryorder='category descending', label="yr3")

gender3_dim = go.parcats.Dimension(values=df.yr4_behavior2, categoryorder='category descending', label="yr4")


#survival_dim = go.parcats.Dimension(
  #  values=df.behavior_yr2, label="yr2", 
 #   #categoryarray=[0, 1],
  #  ticktext=['perished', 'survived'])

# Create parcats trace
#color = df.behavior_yr1;
#colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, gender1_dim, gender2_dim, gender3_dim],
 #       line={'color': color, 'colorscale': colorscale},
      #  hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [36]:
df=donations.groupby(['cohort', 'yr1_behavior2'
                 #'yr3_behavior2', 'yr4_behavior2'
                ]).count().reset_index()
fig = px.parallel_categories(df, dimensions=['cohort', 'yr1_behavior2'],
                color="cohort_gift_pattern", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

In [53]:
df

,cohort,yr1_behavior2,id,first_gift_year,lapsed_count,recovered_count,max_consec_giving_years,total_giving_years,count_2013,count_2014,...,yr2_behavior,yr3_behavior,yr4_behavior,yr0_behavior2,yr2_behavior2,yr3_behavior2,yr4_behavior2,first_gift_year_year,donation_counts,yr0_amount_groups
0,cohort13,did_not_donate,257939,257939,257939,257939,257939,257939,257939,257939,...,257939,257939,257939,257939,257939,257939,257939,257939,257939,257939
1,cohort13,downgrade,23847,23847,23847,23847,23847,23847,23847,23847,...,23847,23847,23847,23847,23847,23847,23847,23847,23847,23847
2,cohort13,maintain,11171,11171,11171,11171,11171,11171,11171,11171,...,11171,11171,11171,11171,11171,11171,11171,11171,11171,11171
3,cohort13,upgrade,27112,27112,27112,27112,27112,27112,27112,27112,...,27112,27112,27112,27112,27112,27112,27112,27112,27112,27112


In [52]:
df=donations.query('cohort=="cohort13"').groupby(['cohort', 'yr1_behavior2'
                 #'yr3_behavior2', 'yr4_behavior2'
                ]).count().reset_index()
fig = px.parallel_categories(df, dimensions=['cohort', 'yr1_behavior2'],
                color="amount_yr1", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

In [37]:
df=c13b.groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2',
                 #'yr3_behavior2', 'yr4_behavior2'
                ]).count().reset_index()
fig = px.parallel_categories(df, dimensions=['cohort', 'yr1_behavior2', 'yr2_behavior2'],
                color="cohort_gift_pattern", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

In [38]:
df=c13b.groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2', 'yr3_behavior2',
                 #'yr4_behavior2'
                ]).count().reset_index()
fig = px.parallel_categories(df, dimensions=['cohort', 'yr1_behavior2', 'yr2_behavior2', 'yr3_behavior2'],
                color="cohort_gift_pattern", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

In [115]:
new_df=donations.query('cohort=="cohort13" & yr4_TF_accum !="TFFFF"').groupby(['cohort','yr0_TF','yr1_TF', 'yr2_TF', 'yr3_TF', 'yr4_TF','yr4_TF_accum']).count().reset_index()

In [116]:
new_df['perc']=new_df['id']/new_df['id'].sum()

In [117]:
new_df

,cohort,yr0_TF,yr1_TF,yr2_TF,yr3_TF,yr4_TF,yr4_TF_accum,id,first_gift_year,lapsed_count,...,yr4_behavior,yr0_behavior2,yr1_behavior2,yr2_behavior2,yr3_behavior2,yr4_behavior2,first_gift_year_year,donation_counts,yr0_amount_groups,perc
0,cohort13,T,F,F,F,T,TFFFT,5010,5010,5010,...,5010,5010,5010,5010,5010,5010,5010,5010,5010,0.054302
1,cohort13,T,F,F,T,F,TFFTF,6773,6773,6773,...,6773,6773,6773,6773,6773,6773,6773,6773,6773,0.073411
2,cohort13,T,F,F,T,T,TFFTT,2195,2195,2195,...,2195,2195,2195,2195,2195,2195,2195,2195,2195,0.023791
3,cohort13,T,F,T,F,F,TFTFF,10390,10390,10390,...,10390,10390,10390,10390,10390,10390,10390,10390,10390,0.112615
4,cohort13,T,F,T,F,T,TFTFT,1453,1453,1453,...,1453,1453,1453,1453,1453,1453,1453,1453,1453,0.015749
5,cohort13,T,F,T,T,F,TFTTF,2412,2412,2412,...,2412,2412,2412,2412,2412,2412,2412,2412,2412,0.026143
6,cohort13,T,F,T,T,T,TFTTT,1898,1898,1898,...,1898,1898,1898,1898,1898,1898,1898,1898,1898,0.020572
7,cohort13,T,T,F,F,F,TTFFF,28817,28817,28817,...,28817,28817,28817,28817,28817,28817,28817,28817,28817,0.312342
8,cohort13,T,T,F,F,T,TTFFT,2196,2196,2196,...,2196,2196,2196,2196,2196,2196,2196,2196,2196,0.023802
9,cohort13,T,T,F,T,F,TTFTF,3075,3075,3075,...,3075,3075,3075,3075,3075,3075,3075,3075,3075,0.033329


In [118]:
df=new_df
fig = px.parallel_categories(df, dimensions=['yr1_TF', 'yr2_TF', 'yr3_TF', 'yr4_TF'],
                color="perc", 
                             color_continuous_scale=px.colors.sequential.Greens,
             #   labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'}
                            )
fig.show()

In [119]:
df=new_df.groupby(['cohort','yr0_TF','yr1_TF','yr2_TF']).sum()[['id','perc']].reset_index().sort_values(['yr1_TF','yr2_TF'], ascending=[False,False])

In [120]:
df

,cohort,yr0_TF,yr1_TF,yr2_TF,id,perc
3,cohort13,T,T,T,26080,0.282676
2,cohort13,T,T,F,36050,0.390739
1,cohort13,T,F,T,16153,0.175079
0,cohort13,T,F,F,13978,0.151505


In [139]:
pd.Series(df.groupby('yr2_TF').sum()[['perc']]['perc']).tolist() + pd.Series(df['perc']).tolist()

[0.542244285234281,
 0.457755714765719,
 0.2826763204387553,
 0.3907393156371598,
 0.17507939432696373,
 0.1515049695971212]

In [140]:
all_nodes1

['cohort13', 'cohort13', 'cohort13', 'cohort13', 'T', 'F', 'T', 'F']

In [145]:
source_indices1

[0, 0, 0, 0, 4, 5, 4, 5]

In [146]:
[all_nodes1.index(yr2_TF) for yr2_TF in df.yr2_TF]+[all_nodes1.index(yr1_TF) for yr1_TF in df.yr1_TF]

[4, 5, 4, 5, 4, 4, 5, 5]

In [147]:
target_indices1

[4, 5, 4, 5, 6, 7, 6, 7]

In [153]:
### THIS ONE WORKS

all_nodes1 = ['cohort13', 'cohort13', 'T', 'F']
#df.cohort.values.tolist() + df.yr2_TF.values.tolist() 
#+ df.yr2_behavior2.values.tolist() 
source_indices1 = [0, 1, 2]
[all_nodes1.index(cohort) for cohort in df.cohort]+[all_nodes1.index(yr2_TF) for yr2_TF in df.yr2_TF]
#[all_nodes1.index(cohort) for cohort in df.cohort] + [all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]
target_indices1 = [1,2,3,4,3,4]
#[all_nodes1.index(yr2_TF) for yr2_TF in df.yr2_TF]+[6,6,7,7]
#[all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]+

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      label =  all_nodes1,
      color =  "red"
    ),

    # Add links
    link = dict(
      source =  source_indices1,
      target =  target_indices1,
      value = pd.Series(df.groupby('yr2_TF').sum()[['perc']]['perc']).tolist() + pd.Series(df['perc']).tolist()
    #    [36,13,52,11,3.42,21.13,2,6.23,4.3,32.49,2.42,17.03],
      #  df['count'],
))])

#value = df.groupby(['Week', 'Sales Bucket'])['Id'].agg('sum').values.flatten()


fig.update_layout(title_text="Behavior of Donors in Yr1 and Yr2",
                  font_size=10)
fig.show()

In [121]:
df
fig = px.parallel_categories(df, dimensions=['yr0_TF','yr1_TF', 'yr2_TF'],
                color="perc", 
                             color_continuous_scale=px.colors.sequential.Greens,
             #   labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'}
                            )
fig.show()

In [88]:
df=c13b.groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2',
                 #'yr3_behavior2', 'yr4_behavior2'
                ]).count().reset_index()
# Create dimensions
class_dim = go.parcats.Dimension(
    values=df.cohort,
    categoryorder='category ascending', label=""
)

gender_dim = go.parcats.Dimension(values=df.yr1_behavior2, categoryorder='category descending', label="yr1")

gender1_dim = go.parcats.Dimension(values=df.yr2_behavior2, categoryorder='category descending', label="yr2")

#gender2_dim = go.parcats.Dimension(values=df.yr3_behavior2, categoryorder='category descending', label="yr3")

#gender3_dim = go.parcats.Dimension(values=df.yr4_behavior2, categoryorder='category descending', label="yr4")


#survival_dim = go.parcats.Dimension(
  #  values=df.behavior_yr2, label="yr2", 
 #   #categoryarray=[0, 1],
  #  ticktext=['perished', 'survived'])

# Create parcats trace
#color = df.behavior_yr1;
#colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, gender1_dim, 
                                               #gender2_dim, gender3_dim
                                              ],
 #       line={'color': color, 'colorscale': colorscale},
      #  hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [16]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Nodes & links
nodes = [['ID', 'Label', 'Color'],
        [0,'AKJ Education','#4994CE'],
        [1,'Amazon','#8A5988'],
        [2,'Flipkart','#449E9E'],
        [3,'Books','#7FC241'],
        [4,'Computers & tablets','#D3D3D3'],
        [5,'Other','#4994CE'],]

# links with your data
links = [['Source','Target','Value','Link Color'],

        # AKJ
        [0,3,1,'rgba(127, 194, 65, 0.2)'],
        [0,4,1,'rgba(127, 194, 65, 0.2)'],

        # Amazon
        [1,3,1,'rgba(211, 211, 211, 0.5)'],
        [1,4,1,'rgba(211, 211, 211, 0.5)'],
        [1,5,1,'rgba(211, 211, 211, 0.5)'],

        # Flipkart
        [2,5,1,'rgba(253, 227, 212, 1)'],
        [2,3,1,'rgba(253, 227, 212, 1)'],]

nodes_headers = nodes.pop(0)
links_headers = links.pop(0)
df_nodes = pd.DataFrame(nodes, columns = nodes_headers)
df_links = pd.DataFrame(links, columns = links_headers)

# Sankey plot setup
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
    # thickness = 30,
      line = dict(
        color = "black",
        width = 0
      ),
      label =  df_nodes['Label'].dropna(axis=0, how='any'),
      color = df_nodes['Color']
    ),
    link = dict(
      source = df_links['Source'].dropna(axis=0, how='any'),
      target = df_links['Target'].dropna(axis=0, how='any'),
      value = df_links['Value'].dropna(axis=0, how='any'),
      color = df_links['Link Color'].dropna(axis=0, how='any'),
  )
)

layout = dict(
        title = "Draw Sankey Diagram from dataframes",
    height = 772,
    font = dict(
      size = 10),)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)

In [21]:
donationsi=donations[['id', 'cohort', 'yr0_behavior2', 'yr1_behavior2', 'yr2_behavior2', 'yr3_behavior2', 'yr4_behavior2']].copy()

In [22]:
donationsii=donationsi.query('cohort=="cohort13" & yr1_behavior2 != "did_not_donate"').groupby(['cohort', 'yr0_behavior2', 'yr1_behavior2']).count()['id'].reset_index()

In [23]:
donationsii['count']=donationsii['id']/donationsii['id'].sum()

In [24]:
donationsii

,cohort,yr0_behavior2,yr1_behavior2,id,count
0,cohort13,n/a,downgrade,23847,0.383824
1,cohort13,n/a,maintain,11171,0.179800
2,cohort13,n/a,upgrade,27112,0.436375


In [25]:
donationsiii=donationsi.query('cohort=="cohort13" & yr1_behavior2 != "did_not_donate" & yr2_behavior2 != "did_not_donate"').groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2']).count()['id'].reset_index()
#var_name='yr', val_name='behavior'

In [26]:
donationsiii['count']=donationsiii['id']/donationsiii['id'].sum()

In [27]:
donationsiii.drop('id', axis=1, inplace=True)

In [28]:
donationsiv=donationsiii.query('yr1_behavior2 != "upgrade"')

In [29]:
pd.Series(donationsiii.groupby('yr1_behavior2').sum()[['count']]['count']).tolist()

[0.35548312883435584, 0.12511503067484664, 0.5194018404907976]

In [30]:
pd.Series(donationsiii['count']).tolist()

[0.11004601226993865,
 0.03416411042944785,
 0.21127300613496933,
 0.019746932515337424,
 0.062346625766871167,
 0.04302147239263804,
 0.32488496932515337,
 0.024233128834355827,
 0.17028374233128835]

In [31]:
donationsiii

,cohort,yr1_behavior2,yr2_behavior2,count
0,cohort13,downgrade,downgrade,0.110046
1,cohort13,downgrade,maintain,0.034164
2,cohort13,downgrade,upgrade,0.211273
3,cohort13,maintain,downgrade,0.019747
4,cohort13,maintain,maintain,0.062347
5,cohort13,maintain,upgrade,0.043021
6,cohort13,upgrade,downgrade,0.324885
7,cohort13,upgrade,maintain,0.024233
8,cohort13,upgrade,upgrade,0.170284


In [33]:
all_nodes = donationsii.cohort.values.tolist() + donationsii.yr1_behavior2.values.tolist()
source_indices = [all_nodes.index(cohort) for cohort in donationsii.cohort]
target_indices = [all_nodes.index(yr1_behavior2) for yr1_behavior2 in donationsii.yr1_behavior2]

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      label =  all_nodes,
      color =  "red"
    ),

    # Add links
    link = dict(
      source =  source_indices,
      target =  target_indices,
      value =  donationsii['count'],
))])

fig.update_layout(title_text="Behavior of Donors Retained in Yr1",
                  font_size=10)
fig.show()

In [34]:
target_indices

[3, 4, 5]

In [53]:
[all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]

[9, 9, 9, 12, 12, 12, 15, 15, 15]

In [75]:
pd.Series(donationsiii.groupby('yr1_behavior2').sum()[['count']]['count']).tolist()

[0.35548312883435584, 0.12511503067484664, 0.5194018404907976]

In [37]:
[all_nodes1.index(yr2_behavior2) for yr2_behavior2 in donationsiii.yr2_behavior2]


[9, 12, 15, 9, 12, 15, 9, 12, 15]

In [38]:
all_nodes1

['cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'downgrade',
 'downgrade',
 'downgrade',
 'maintain',
 'maintain',
 'maintain',
 'upgrade',
 'upgrade',
 'upgrade']

In [39]:
source_indices1

[0, 0, 0, 9, 9, 9, 12, 12, 12, 15, 15, 15]

In [40]:
target_indices1

[9, 12, 15, 18, 21, 24, 18, 21, 24, 18, 21, 24]

In [92]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["c13", 'down', "main", "up", 'down1', "main1", "up1"],
      color = "blue"
    ),
    link = dict(
      source = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = [1, 2, 3, 4, 5, 6, 4, 5, 6, 4, 5, 6],
      value = [36,13,52,11,3.42,21.13,2,6.23,4.3,32.49,2.42,17.03]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [44]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Original data
data = [['Source','Target','Value','Color','Node, Label','Link Color'],
    [0,5,20,'#F27420','Remain+No – 28','rgba(253, 227, 212, 0.5)'],
    [0,6,3,'#4994CE','Leave+No – 16','rgba(242, 116, 32, 1)'],
    [0,7,5,'#FABC13','Remain+Yes – 21','rgba(253, 227, 212, 0.5)'],
    [1,5,14,'#7FC241','Leave+Yes – 14','rgba(219, 233, 246, 0.5)'],
    [1,6,1,'#D3D3D3','Didn’t vote in at least one referendum – 21','rgba(73, 148, 206, 1)'],
    [1,7,1,'#8A5988','46 – No','rgba(219, 233, 246,0.5)'],
    [2,5,3,'#449E9E','39 – Yes','rgba(250, 188, 19, 1)'],
    [2,6,17,'#D3D3D3','14 – Don’t know / would not vote','rgba(250, 188, 19, 0.5)'],
    [2,7,2,'','','rgba(250, 188, 19, 0.5)'],
    [3,5,3,'','','rgba(127, 194, 65, 1)'],
    [3,6,9,'','','rgba(127, 194, 65, 0.5)'],
    [3,7,2,'','','rgba(127, 194, 65, 0.5)'],
    [4,5,5,'','','rgba(211, 211, 211, 0.5)'],
    [4,6,9,'','','rgba(211, 211, 211, 0.5)'],
    [4,7,8,'','','rgba(211, 211, 211, 0.5)']
    ]



headers = data.pop(0)
df = pd.DataFrame(data, columns = headers)
scottish_df = df

data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0
      ),
      label =  scottish_df['Node, Label'].dropna(axis=0, how='any'),
      color = scottish_df['Color']
    ),
    link = dict(
      source = scottish_df['Source'].dropna(axis=0, how='any'),
      target = scottish_df['Target'].dropna(axis=0, how='any'),
      value = scottish_df['Value'].dropna(axis=0, how='any'),
      color = scottish_df['Link Color'].dropna(axis=0, how='any'),
  )
)

layout =  dict(
    title = "Scottish Referendum Voters who now want Independence",
    height = 772,
    font = dict(
      size = 10
    ),    
)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)

In [45]:
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()

In [93]:
### THIS ONE WORKS
df=donationsiii
all_nodes1 = df.cohort.values.tolist() + df.yr1_behavior2.values.tolist() 
#+ df.yr2_behavior2.values.tolist() 
source_indices1 = [0,0,0] + [all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]
#[all_nodes1.index(cohort) for cohort in df.cohort] + [all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]
target_indices1 = [9,12,15]+(pd.Series([all_nodes1.index(yr2_behavior2) for yr2_behavior2 in df.yr2_behavior2])+ 9).tolist() 
#[all_nodes1.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]+

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      label =  all_nodes1,
      color =  "red"
    ),

    # Add links
    link = dict(
      source =  source_indices1,
      target =  target_indices1,
      value = pd.Series(donationsiii.groupby('yr1_behavior2').sum()[['count']]['count']).tolist() + pd.Series(donationsiii['count']).tolist()
    #    [36,13,52,11,3.42,21.13,2,6.23,4.3,32.49,2.42,17.03],
      #  df['count'],
))])

#value = df.groupby(['Week', 'Sales Bucket'])['Id'].agg('sum').values.flatten()


fig.update_layout(title_text="Behavior of Donors in Yr1 and Yr2",
                  font_size=10)
fig.show()

In [95]:
donationsi

,id,cohort,yr0_behavior2,yr1_behavior2,yr2_behavior2,yr3_behavior2,yr4_behavior2
0,00000ce845c00cbf0686c992fc369df4,cohort13,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
1,00002783bc5d108510f3f9666c8b1edd,cohort16,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2,00002d44003ed46b066607c5455a999a,cohort16,n/a,upgrade,did_not_donate,did_not_donate,did_not_donate
3,00002eb25d60a09c318efbd0797bffb5,cohort15,n/a,maintain,upgrade,downgrade,did_not_donate
4,0000300773fe015f870914b42528541b,cohort17,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
...,...,...,...,...,...,...,...
2024549,ffffb91addfb35f11b30c2943f349654,cohort17,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2024550,ffffc2e1225ac40a3ace9b8562115c9b,cohort16,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2024551,ffffd8f0fbfa10b99efe90a5ee13c5df,cohort17,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate
2024552,ffffdb40226605870f62422b2b420aab,cohort17,n/a,did_not_donate,did_not_donate,did_not_donate,did_not_donate


In [119]:
donationsv=donationsi.query('yr1_behavior2 != "did_not_donate" & yr2_behavior2 != "did_not_donate" & yr3_behavior2 != "did_not_donate" & yr4_behavior2 != "did_not_donate"').groupby(['cohort', 'yr1_behavior2', 'yr2_behavior2','yr3_behavior2','yr4_behavior2']).count()['id'].reset_index()
#var_name='yr', val_name='behavior'


In [120]:
donationsv['count']=donationsv['id']/donationsv['id'].sum()

In [122]:
donationsv.to_csv('donationsv.csv')

In [126]:
donationsi.cohort.value_counts()

cohort16    445598
cohort17    434991
cohort15    355137
cohort14    320673
cohort13    320069
cohort18    147945
cohort12       141
Name: cohort, dtype: int64

In [102]:
[0,0,0] + [all_nodes2.index(yr2_behavior2) for yr2_behavior2 in df.yr2_behavior2]

[0,
 0,
 0,
 27,
 27,
 27,
 36,
 36,
 36,
 45,
 45,
 45,
 27,
 27,
 27,
 36,
 36,
 36,
 45,
 45,
 45,
 27,
 27,
 27,
 36,
 36,
 36,
 45,
 45,
 45]

In [108]:
all_nodes2

['cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'cohort13',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'maintain',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'upgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'maintain',
 'maintain',
 'maintain',
 'upgrade',
 'upgrade',
 'upgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'maintain',
 'maintain',
 'maintain',
 'upgrade',
 'upgrade',
 'upgrade',
 'downgrade',
 'downgrade',
 'downgrade',
 'maintain',
 'maintai

In [110]:
### THIS ONE WORKS
df=donationsv
all_nodes2 = df.cohort.values.tolist() + df.yr1_behavior2.values.tolist() + df.yr2_behavior2.values.tolist() 
#+ df.yr2_behavior2.values.tolist() 
source_indices2 = [0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6]
#[0,0,0] + [all_nodes2.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]
#[all_nodes2.index(cohort) for cohort in df.cohort] + [all_nodes2.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]
target_indices2 = [1,2,3,4,5,6,4,5,6,4,5,6,7,8,9,7,8,9,7,8,9,7,8,9,7,8,9,7,8,9,7,8,9,7,8,9,7,8,9]
#[9,12,15]+(pd.Series([all_nodes2.index(yr2_behavior2) for yr2_behavior2 in df.yr2_behavior2])+ 9).tolist() 
#[all_nodes2.index(yr1_behavior2) for yr1_behavior2 in df.yr1_behavior2]+

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      label =  all_nodes1,
      color =  "red"
    ),

    # Add links
    link = dict(
      source =  source_indices2,
      target =  target_indices2,
      value = pd.Series(df.groupby('yr1_behavior2').sum()[['count']]['count']).tolist() + pd.Series(df['count']).tolist()
    #    [36,13,52,11,3.42,21.13,2,6.23,4.3,32.49,2.42,17.03],
      #  df['count'],
))])

#value = df.groupby(['Week', 'Sales Bucket'])['Id'].agg('sum').values.flatten()


fig.update_layout(title_text="Behavior of Donors in Yr1 and Yr2",
                  font_size=10)
fig.show()